<a href="https://colab.research.google.com/github/jaunty0216/SomeAttempts/blob/main/%E6%96%87%E5%AD%97%E9%9B%B2%E8%A3%BD%E9%80%A0%E5%AE%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jieba

In [3]:
def Paintit():
  from google.colab import files
  uploaded = files.upload()

  for filename,_ in uploaded.items():
    filepath = f"/content/{filename}"
    print(f"Filepath: {filepath}")

  import pandas as pd
  import jieba
  OriginFile = pd.read_table(filepath)
  Origin_df = pd.DataFrame(OriginFile,columns=['contents'])

  for _, content in Origin_df.iterrows():
    word = content['contents']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)